In [1]:
from pathlib import Path, PurePath
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode
from analysis import read_behaviors, read_moea_fitness, unpickle_moea
from utility.util_functions import calc_str_disparity, euclidean_distance
from utility.visualize import plot_path, plot_moea_fitness, plot_surrogate_model, plot_str_disparity, plot_moea_fitness_2d
import pickle
import numpy as np
from deap import base, creator
from evolution.simulation import Disparity
import scipy as sp
from scipy import stats
import math
init_notebook_mode(connected=True)


ModuleNotFoundError: DBus works only on linux!


Using TensorFlow backend.


ModuleNotFoundError: DBus works only on linux!


In [2]:
# get the path for all behavioral features
files = sorted(Path('data/results/').glob('transferability_*/behavioral_features.dat'))
fitness_f = sorted(Path('data/results/').glob('transferability_*/fitness.dat'))

In [3]:
columns = [
        'avg_left', 'avg_right',
        's1', 's2', 's3', 's4', 's5', 's6', 's7',
        'area0_percentage',
        'area1_percentage',
        'area2_percentage',
    ]

In [4]:
fitness_f[10]

PosixPath('data/results/transferability_simulation_19/fitness.dat')

In [5]:
behaviors = read_behaviors([files[10]])[0]

In [ ]:
behaviors.loc[behaviors['genome_id'] == 494].head()

In [6]:
behaviors.head()

,gen,genome_id,simulation,avg_left,avg_right,s1,s2,s3,s4,s5,s6,s7,area0_percentage,area1_percentage,area2_percentage
0,0,1,VREP,0.440869,0.750711,0.0,0.0,0.0,0.0,0.0,0.99000,0.99000,1.0,0.0,0.0
1,0,1,VREP,0.692531,0.754626,0.0,0.0,0.0,0.0,0.0,0.13000,0.10000,1.0,0.0,0.0
2,0,1,THYMIO,0.685312,0.753420,0.0,0.0,0.0,0.0,0.0,0.22449,0.22449,1.0,0.0,0.0
3,0,1,VREP,0.692531,0.754626,0.0,0.0,0.0,0.0,0.0,0.13000,0.10000,1.0,0.0,0.0
4,0,2,VREP,0.102864,0.755891,0.0,0.0,0.0,0.0,0.0,0.30000,0.11000,1.0,0.0,0.0


In [ ]:
b_grouped = behaviors.groupby(['genome_id']).filter(lambda x: len(x) > 1).groupby(['genome_id', 'simulation']).first()

In [ ]:
b_grouped.head()

In [ ]:
euclidean_distance(b_grouped[columns].iloc[1], b_grouped[columns].iloc[0])

In [ ]:
for i in range(1, len(b_grouped), 2):
    print('genome_id:', 
          b_grouped[columns].iloc[i].name[0],
          'actual diversity',
          euclidean_distance(
              b_grouped[columns].iloc[i],
              b_grouped[columns].iloc[i-1]
          )
    )

In [7]:
fitness_data = read_moea_fitness([fitness_f[10]])[0]

In [ ]:
fitness_data.sort_values(by=['fitness'])

In [8]:
fitness_data.head()

,gen,genome_id,fitness,str_disparity,diversity
0,0,1,0.557418,0.046722,1.0
1,0,2,0.044272,0.046722,1.0
2,0,3,0.384538,0.046722,1.0
3,0,4,0.304699,0.046722,1.0
4,0,5,0.317067,0.046722,1.0


In [9]:
hof = unpickle_moea('data/results/transferability_simulation_19/hof')

In [10]:
len(hof)

5

In [11]:
plot_moea_fitness_2d(fitness_data, hof)

/Users/richban/.local/share/virtualenvs/behavioral.neuroevolution-ViNSkuNA/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [13]:
hof_ids = []
for ind in hof:
    hof_ids.append(ind.key)
    print(ind.key, ind.fitness.values)

125 (8.744681767627817, 0.5122868791270172, 1.0)
82 (7.684711082733685, 0.4617108847122681, 1.0)
78 (6.019636037850952, 0.3853453848548356, 1.0)
55 (5.515731964375675, 0.23182525509237725, 1.0)
32 (2.7182724817528037, 0.046721524445696314, 1.0)


In [12]:
genomes_file = sorted(Path('data/results/transferability_simulation_19/deap_inds').glob('*_genome.pkl'))
transfered_file = sorted(Path('data/results/transferability_simulation_19/deap_inds').glob('*_transformed_genome_.pkl'))
genomes = [unpickle_moea(genome) for genome in genomes_file]
transferred = [unpickle_moea(genome) for genome in transfered_file]

/Users/richban/.local/share/virtualenvs/behavioral.neuroevolution-ViNSkuNA/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning:

A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.

/Users/richban/.local/share/virtualenvs/behavioral.neuroevolution-ViNSkuNA/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning:

A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.



In [ ]:
transfered_file

In [14]:
len(transfered_file)

77

In [47]:
g = sorted(genomes, key=lambda x: x.key)

In [48]:
t = sorted(transferred, key=lambda x: x.key)

In [49]:
sim_genome = list(filter(lambda x: x.key == 55, g))[0]

In [50]:
print(sim_genome.key, sim_genome.fitness.values, sim_genome.sim_time, sim_genome.diversity)

55 (5.515731964375675, 0.23182525509237725, 1.0) 20.00000000000015 0.48842727989685447


In [39]:
trans_genome = list(filter(lambda x: x.key == 82, t))[0]

In [40]:
print(trans_genome.key, trans_genome.fitness.values)

82 (7.684711082733685, 0.4617108847122681, 1.0)


In [41]:
sim_genome.sim_time

30.7000000000003

In [ ]:
sim_genomes = sorted(list(filter(lambda x: x.key in hof_ids[:-10], g)), key=lambda x: x.fitness.values[0])
trans_genomes = sorted(list(filter(lambda x: x.key in hof_ids[:-10] t)), key=lambda x: x.fitness.values[0])

In [42]:
plot_path([sim_genome, trans_genome], 'The discrepancy of traveled trajectories in simulation & reality.')
# plot_path(sim_genomes)

In [ ]:
sim_genome.position

In [ ]:
trans_genome.position

In [ ]:
plot_path(trans_genomes)

In [51]:
calc_str_disparity(trans_genome.position, sim_genome.position)

7.038882699804451

In [24]:
euclidean_distance(trans_genome.position, sim_genome.position)

1.1722152867371614

In [25]:
plot_surrogate_model(fitness_data)

In [26]:
# merge  
union_t = [(x, y) for x in g for y in t if x.key == y.key]
for s, t in union_t:
    np.array_equal(s, t)
    assert s.key == t.key

In [ ]:
str_disparity = Disparity()

In [ ]:
for s, t in union_t:
    print(s.gen, end=" ")

In [27]:
str_disparities = [(s.gen,
                    s.key,
                    s.fitness.values[0],
                    s.fitness.values[1],
                    calc_str_disparity(t.position, s.position)
                   ) for s, t in union_t
                  ]    

str_euclidean = [(s.gen,
                    s.key,
                    s.fitness.values[0],
                    s.fitness.values[1],
                    calc_str_disparity(t.position, s.position)
                   ) for s, t in union_t
                ]

In [ ]:
sorted(str_disparities, key=lambda x: x[0])

In [ ]:
np.array([str_disparity[1] for str_disparity in str_disparities])
np.array([real_disparity[2] for real_disparity in str_disparities])

filter_disparities = list(filter(lambda x: x[2] < 5, str_disparities))
filter_disparities[:4]

In [28]:
plot_str_disparity(str_disparities)

In [29]:
actual = []
approximated = []
for s, t in union_t:
    approximated.append(s.fitness.values[1])
    actual.append(calc_str_disparity(t.position, s.position))

In [30]:
stats.pearsonr(actual, approximated)

(0.2513019485283815, 0.028541518512223498)

In [ ]:
str_euclidean

In [ ]:
distance = 0
for i in range(0, len(trans_genome.position)-1):
    distance += math.sqrt(((sim_genome.position[i][0] - 
                            sim_genome.position[i][1])**2) + 
                          ((sim_genome.position[i+1][1] - 
                            sim_genome.position[i+1][1])**2))

In [32]:
def traveled_distance(genome):
    distance = 0
    for i in range(0, len(genome.position)-1):
        distance += math.sqrt(((genome.position[i][0] - 
                                genome.position[i][1])**2) + 
                              ((genome.position[i+1][1] - 
                                genome.position[i+1][1])**2))
    return distance

In [33]:
traveled_distance(trans_genome)

10.263571097805974

In [34]:
traveled_distance(sim_genome)

8.587992548942566